In [0]:
datapath= "gdrive/My Drive/data/rdyn_data/results_4000/"
datapath_s = "gdrive/My Drive/data/"
ilcd_path =  "gdrive/My Drive/data/iLCD_share/"
algorithm_name = "ilcd"
os.chdir("/content")

! pip install nf1

#res_fn = os.listdir(datapath+data_folder)




In [0]:


datapath = "gdrive/My Drive/data/rdyn_data_6/"
data_folder= "results/"
datapathoutput = "gdrive/My Drive/data/iLCD_share/"
algorithm_name = 'ilcd'

idc = datapath + '/4000_100_15_0.6_0.8_0.2_1_3.5_90 2'
gtc = datapath + '/4000_100_15_0.6_0.8_0.2_1_3.5_90'

#rdyn_instance= "2000_100_15_0.6_0.8_0.2_1_2.5_0"
import os
import time
import pandas as pd
from nf1 import NF1
#results_fn = listdir van rdyn_instances

res_fn = os.listdir(datapath+data_folder)

print (res_fn)


def read_coms(filename):
        com = []
        with open(filename) as f:
            for l in f:
                try:
                    l = l.replace("(","[").replace(")","]").replace("]", "").replace(" ", "").split("[")[1]
                except:
                    pass
                ns = tuple(map(int, l.rstrip().replace(" ", "\t").replace(",", "\t").split("\t")))

                if len(ns) == 2:
                    raise Exception
                com.append(ns)
        return com

      
      
      
rows_list= []
for rdyn_instance in res_fn:  
    
    # get gt comms
    fn = os.listdir(datapath+data_folder+rdyn_instance)
    
    comms = [x for x in fn if 'communities' in x] 
    #print(comms)
    print(rdyn_instance, len(comms))
    #print(comms)
    
    
    comm_nr = []
    for c in comms:
        comm_nr.append(int(c[12:].replace(".txt",''),))
    #print (sorted(comm_nr))
    # get identified comms
    #i_comms = os.listdir(datapathoutput+"/"+rdyn_instance+"/output_olcpm.txt")#output_folder + rdyn_instance)
    #print (i_comms)
    #id_comms = [x for x in i_comms if "AlvCom" in x]

    
  
    #gt_index = [[int(x[10:-4]),x] for x in id_comms]
    
 
    #print ("gt_index",gt_index)
    #print("id_comms", id_comms)
    
    
    count = 0
    # voor elke gt community
    for nr in comm_nr:
      with open(datapathoutput+rdyn_instance+"/interactions_f.lnd.tcom")as f:# output_folder + rdyn_instance + "/"+v) as f:
          #print(idc+"/interactions_f.lnd.tcom")
          identified_coms = []
          com = []
          for l in f:
              if l[0] =='#':
                break
              
              #line
              #line[2]= l.replace("(","[").replace(")","]").replace("]", "").replace(" ", "").split("[")[1]
              line = l.split('\t')
              line = line[1:]
              key = line[0][:line[0].find(":")]
              #print(line[0][:line[0].find(":")], nr)
              lin = [int (y) for y in line[1:-1]]
              #print (key, lin, nr,)
              if int(nr) == int(key):
                
                  identified_coms.append(tuple(lin))
          #break        
                  #print(identified_coms)
          gt = read_coms(datapath+data_folder+rdyn_instance+"/communities-"+str(nr)+".txt")
          #print(gt)
          #break
          #count +=1
          #print(c,datapath+data_folder+rdyn_instance+'/'+comms[c])






          if identified_coms == []:
                  continue
          #print(output_folder + rdyn_instance + "/"+v)
          #print(comm_nr)
          #print(comms)





          nf = NF1(identified_coms, gt)
          #print(nf.summary())
          res = []
          res.extend(nf.get_f1())
          #print(nf.get_partition_stats())
          #print ()
          mean = str(res[0])
          std = str(res[1])
          maxx = str(res[2])
          minn = str(res[3])
          mode =str(res[4])


          row = {
              "algorithm": algorithm_name,
              "rdyn_instance": str(rdyn_instance),
              #"graph_n": str(alg_results_fn[i]),
              "mean": mean.replace(".",","),
              "std" : std.replace(".",","),
              "min": minn.replace(".",","),
              "max": maxx.replace(".",","),
              "mode": mode.replace(".",",")
          }
          rows_list.append(row)
      #print(rows_list)
      #print(os.getcwd())
      #else:
          #print('skip')

df = pd.DataFrame(rows_list)  
#print(datapath+"output_4000_"+algorithm_name+".txt")
df.to_csv(datapath+"output_2000_t_"+algorithm_name+".txt", sep=' ', index=False)


In [0]:
import os


#for elm in res_fn:
   # print(elm)

i = "/interactions.txt"
import shutil
import pandas as pd
src_f = '/source'
dest_f = '/destination'

rdyns = os.listdir(datapath)
import time
start_global = time.time()
times = []    

for _, rdyn in enumerate(rdyns):
    print(_, rdyn)
    if _ > 49:
      break
    #src = datapath+rdyn+"/interactions.txt"
    #dst = ilcd_path+src_f+"/interactions_"+rdyn+".txt"
    #shutil.copy(src, dst)
    
    print(os.getcwd())
    data = pd.read_csv(datapath + rdyn + i , sep="\t", header=None)
    data.columns = ["iteration", "timestamp", "action", "node_id1", "node_id2"]
    cols = data.columns.tolist()
    
    cols = [cols[0], cols[2], cols[3], cols[4]]
    data = data[cols]
    os.mkdir(ilcd_path+rdyn)
    data.to_csv(ilcd_path+rdyn+"/interactions_f.lnd", sep="\t",  index=False, header=False)
   
    
    ##
    
    
    src = "gdrive/My Drive/data/iLCD_share/iLCD.jar"
    dst = ilcd_path+rdyn+ '/iLCD.jar'
    shutil.copy(src, dst)
    
    
    ##
    os.chdir(ilcd_path+rdyn)
    start = time.time()
    !java -jar iLCD.jar -i interactions_f.lnd > /dev/null
    end = time.time()
    x = str(end-start)
        
    run_time = {   "algorithm": algorithm_name,
                   "rdyn_instance_name" : rdyn,
                   "execution_time" : x.replace(".",",") 
    }
        
    os.chdir("/content")
    

    times.append(run_time)

df = pd.DataFrame(times)

df.to_csv(datapath_s + "execution_times_tt1000_"+algorithm_name+".txt", sep=' ', index=False)

end_global = time.time()
print(end_global - start_global)

0 4000_100_15_0.6_0.8_0.2_1_2.5_0
/content
1 4000_100_15_0.6_0.8_0.2_1_3_0
/content
2 4000_100_15_0.6_0.8_0.2_1_3.5_0
/content
3 4000_100_15_0.6_0.8_0.2_1_2.5_1
/content
4 4000_100_15_0.6_0.8_0.2_1_3_1
/content
5 4000_100_15_0.6_0.8_0.2_1_3.5_1
/content
6 4000_100_15_0.6_0.8_0.2_1_2.5_2
/content
7 4000_100_15_0.6_0.8_0.2_1_3_2
/content
8 4000_100_15_0.6_0.8_0.2_1_3.5_2
/content
9 4000_100_15_0.6_0.8_0.2_1_2.5_3
/content
10 4000_100_15_0.6_0.8_0.2_1_3_3
/content
11 4000_100_15_0.6_0.8_0.2_1_3.5_3
/content
12 4000_100_15_0.6_0.8_0.2_1_2.5_4
/content
13 4000_100_15_0.6_0.8_0.2_1_3_4
/content
BE CAREFUL : merge not printed between50_789 and 51_323
14 4000_100_15_0.6_0.8_0.2_1_3.5_4
/content
15 4000_100_15_0.6_0.8_0.2_1_2.5_5
/content
16 4000_100_15_0.6_0.8_0.2_1_3_5
/content
17 4000_100_15_0.6_0.8_0.2_1_2.5_6
/content
18 4000_100_15_0.6_0.8_0.2_1_3_6
/content
19 4000_100_15_0.6_0.8_0.2_1_3.5_6
/content
20 4000_100_15_0.6_0.8_0.2_1_2.5_7
/content
21 4000_100_15_0.6_0.8_0.2_1_3_7
/content
22